<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [15]:
import sys
from pathlib import Path

# Ajouter le dossier src au path
src_path = Path("..") / ".."
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2


import os
import uuid
import json
import mlflow
import tempfile
from datetime import datetime
from pathlib import Path
from huggingface_hub import HfApi
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.utils import safe_mean
from dotenv import load_dotenv
import textwrap
from huggingface_hub import hf_hub_download
from env.snake_env import SnakeEnv

load_dotenv()
hf_token = os.getenv("HF_HUB_TOKEN")
if not hf_token:
    raise ValueError("⚠️ Variable HF_HUB_TOKEN manquante.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3 style="color:#CCC229">TRAIN FUNCTION<h3>

In [58]:
def load_snake_model_data(uuid: str, hf_repo_id: str):
    # 1. Sécurité Token
    token = os.getenv("HF_HUB_TOKEN")
    if not token:
        print("❌ Erreur : HF_HUB_TOKEN manquant.")
        return None, None

    print(f"🔍 Scan du dépôt pour trouver l'UUID : {uuid} ...")
    api = HfApi(token=token)

    try:
        # 2. On récupère la liste COMPLÈTE des fichiers du repo
        # Cela nous donne une liste de chaînes : ["5x5/abc.../model.zip", "10x10/xyz.../metadata.json", ...]
        all_files = api.list_repo_files(repo_id=hf_repo_id, repo_type="model", token=token)

        # 3. On cherche l'aiguille dans la botte de foin
        target_path = None
        for filename in all_files:
            # On cherche le fichier metadata qui contient notre UUID dans son chemin
            if uuid in filename and filename.endswith("metadata.json"):
                target_path = filename
                break

        if not target_path:
            print(f"❌ Impossible de trouver un dossier contenant l'UUID {uuid}")
            return None, None

        # target_path vaut maintenant quelque chose comme : "5x5/20a0ecbd.../metadata.json"
        print(f"📍 Fichier trouvé : {target_path}")

        # 4. On télécharge ce fichier précis
        local_meta_path = hf_hub_download(
            repo_id=hf_repo_id,
            filename=target_path,
            repo_type="model",
            token=token
        )

        # 5. On lit la grid_size (pour être sûr)
        with open(local_meta_path, "r") as f:
            data = json.load(f)
            grid_size = data.get("grid_size")

        # 6. On déduit le chemin du modèle .zip
        # On remplace juste "metadata.json" par "model.zip" dans le chemin qu'on a trouvé
        model_path_in_repo = target_path.replace("metadata.json", "model.zip")

        print(f"📥 Téléchargement du modèle : {model_path_in_repo}")
        local_model_path = hf_hub_download(
            repo_id=hf_repo_id,
            filename=model_path_in_repo,
            repo_type="model",
            token=token
        )

        # 7. Chargement final
        agent = PPO.load(local_model_path)
        print(f"✅ Succès ! Agent chargé (Grille {grid_size}x{grid_size})")

        return agent, grid_size

    except Exception as e:
        print(f"❌ Erreur lors du scan/chargement : {e}")
        return None, None

In [55]:
def list_snake_models(
    grid_size_filter: int = None,
    sort_by: str = "date",  #
    hf_repo_id: str = "Lategardener/snake-rl-models"
):
    print(f"🔍 Recherche des modèles dans {hf_repo_id}...")
    api = HfApi(token=hf_token)

    # 1. Récupérer tous les fichiers du repo
    all_files = api.list_repo_files(repo_id=hf_repo_id, repo_type="model")

    # 2. Filtrer uniquement les fichiers metadata.json
    meta_files = [f for f in all_files if f.endswith("metadata.json")]

    models_data = []

    # 3. Lire chaque fichier metadata (C'est rapide car ce sont de petits fichiers JSON)
    for meta_file in meta_files:
        try:
            # Téléchargement en cache
            local_path = hf_hub_download(repo_id=hf_repo_id, filename=meta_file, repo_type="model", token=hf_token)

            with open(local_path, "r") as f:
                data = json.load(f)

            # Filtrage par taille de grille
            model_grid = data.get("grid_size")
            if grid_size_filter is not None and model_grid != grid_size_filter:
                continue

            # Parsing de la date pour le tri
            date_str = data.get("date", "")
            try:
                dt_object = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")
            except ValueError:
                dt_object = datetime.min

            data["_dt_object"] = dt_object # Stocké pour le tri interne
            models_data.append(data)

        except Exception as e:
            print(f"⚠️ Erreur lecture {meta_file}: {e}")

    # 4. Tri
    if sort_by == "date":
        models_data.sort(key=lambda x: x["_dt_object"], reverse=True) # Plus récent en premier
    elif sort_by == "reward":
        # On met -inf si pas de reward pour qu'ils soient à la fin
        models_data.sort(key=lambda x: x.get("final_mean_reward") or float('-inf'), reverse=True)

    # 5. Affichage Tableau
    print(f"\n{' ' * 12} {'MODELE (UUID)'} {' ' * 17} || {' ' * 15} {'DESCRIPTION'}")
    print("-" * 110)

    for m in models_data:
        uuid_display = f"{m.get('grid_size'):02d}x{m.get('grid_size'):02d} / {m.get('uuid')}"

        # Construction de la description
        algo = m.get('algorithm', 'N/A')
        date = m.get('date', 'N/A')
        rew = m.get('final_mean_reward')
        rew_str = f"{rew:.2f}" if isinstance(rew, (int, float)) else "N/A"

        desc = f"Algo: {algo} | Date: {date} | Reward: {rew_str}"

        print(f"{uuid_display} || {desc}")

    print("-" * 110)
    print(f"✅ Nombre de modèles trouvés : {len(models_data)}")

In [4]:
# ------------------------------------------
# Callback
# ------------------------------------------
class MLflowLoggingCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
    def _on_step(self) -> bool:
        return True
    def _on_rollout_end(self) -> None:
        try:
            logger_values = getattr(self.logger, "name_to_value", {})
            metrics = {k: float(v) for k, v in logger_values.items()}
            if hasattr(self.model, "ep_info_buffer") and len(self.model.ep_info_buffer) > 0:
                metrics["rollout/ep_rew_mean"] = float(safe_mean([ep["r"] for ep in self.model.ep_info_buffer]))
                metrics["rollout/ep_len_mean"] = float(safe_mean([ep["l"] for ep in self.model.ep_info_buffer]))
            if metrics:
                mlflow.log_metrics(metrics, step=self.num_timesteps)
        except Exception:
            pass

In [10]:
# ---------------------------------------------------------
# Wrapper MLflow pour Hugging Face
# ---------------------------------------------------------
class SnakeHFModel(mlflow.pyfunc.PythonModel):
    """
    Classe Wrapper qui permet à MLflow de savoir comment charger et utiliser
    ton modèle stocké sur Hugging Face.
    """
    def __init__(self, repo_id: str, subfolder: str):
        # On sauvegarde les infos pour retrouver le modèle plus tard
        self.repo_id = repo_id
        self.subfolder = subfolder
        self.model = None

    def load_context(self, context):
        # Cette méthode est exécutée quand on charge le modèle via mlflow.pyfunc.load_model()
        from huggingface_hub import hf_hub_download
        from stable_baselines3 import PPO

        print(f"📥 Chargement du contexte modèle depuis {self.repo_id}...")

        # Téléchargement du fichier physique
        model_path = hf_hub_download(
            repo_id=self.repo_id,
            filename=f"{self.subfolder}/model.zip"
        )

        # Chargement en mémoire
        self.model = PPO.load(model_path)

    def predict(self, context, model_input, params=None):
        """
        MLflow exige que le paramètre s'appelle 'model_input'.
        Ici, model_input correspondra à ton observation.
        """
        action, _ = self.model.predict(model_input)
        return action

/home/cytech/programmation/AI/snake_rl/.venv/lib/python3.10/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [12]:
def train_snake(
    timesteps: int = 100_000,
    grid_size: int = None,      # Optionnel si base_uuid est fourni
    n_envs: int = 4,
    algorithm: str = "PPO",
    hf_repo_id: str = "Lategardener/snake-rl-models",
    base_uuid: str = None       # L'UUID du modèle parent (déclenche le fine-tuning)
):
    # --- A. Initialisation ---
    now = datetime.now()
    readable_date = now.strftime("%d/%m/%Y %H:%M:%S")
    date_str = now.strftime("%Y%m%d_%H%M%S")
    new_agent_uuid = str(uuid.uuid4())

    # --- B. Logique de Chargement (Le Cerveau) ---
    agent = None
    is_finetuning = False

    if base_uuid:
        # Mode FINE-TUNING : On va chercher le modèle et ses infos
        try:
            agent, loaded_grid_size = load_snake_model_data(base_uuid, hf_repo_id)

            # On écrase la grid_size avec celle du modèle chargé (c'est la vérité terrain)
            grid_size = loaded_grid_size
            is_finetuning = True
            mode_label = "FINE-TUNING"
            print(f"✅ Modèle chargé avec succès. Reprise sur grille {grid_size}x{grid_size}.")

        except Exception as e:
            print(f"❌ Impossible de charger le modèle {base_uuid} : {e}")
            return # On arrête tout pour ne pas lancer un training vide par erreur
    else:
        # Mode NEW TRAINING
        if grid_size is None:
            raise ValueError("⚠️ Vous devez spécifier 'grid_size' pour un nouvel entraînement.")
        mode_label = "NEW_TRAINING"
        print(f"✨ Création d'un nouvel agent vierge sur grille {grid_size}x{grid_size}.")

    # --- C. Configuration MLflow ---
    run_name = f"{mode_label}_{date_str}_{new_agent_uuid[:8]}"
    mlflow.set_experiment(f"Snake_{grid_size}x{grid_size}")

    print(f"\n🚀 Démarrage Run MLflow : {run_name}")
    print(f"🆔 Nouvel ID : {new_agent_uuid}")

    with mlflow.start_run(run_name=run_name) as run:

        # Tags
        mlflow.set_tag("agent_uuid", new_agent_uuid)
        mlflow.set_tag("hf_repo", hf_repo_id)
        mlflow.set_tag("run_type", "finetuning" if is_finetuning else "fresh")
        if base_uuid:
            mlflow.set_tag("parent_model_uuid", base_uuid)

        # Params
        mlflow.log_params({
            "algorithm": algorithm,
            "grid_size": grid_size,
            "timesteps_added": timesteps,
            "base_model": base_uuid if base_uuid else "None"
        })

        # --- D. Environnement & Agent ---
        env = make_vec_env(lambda: Monitor(SnakeEnv(grid_size=grid_size, render_mode=None)), n_envs=n_envs)

        if is_finetuning:
            agent.set_env(env) # Connexion du vieux cerveau au nouveau corps
        else:
            agent = PPO("MlpPolicy", env, verbose=1)

        # --- E. Entraînement ---
        agent.learn(
            total_timesteps=timesteps,
            callback=MLflowLoggingCallback(),
            reset_num_timesteps=not is_finetuning # False si fine-tuning pour continuer les courbes
        )

        # --- F. Sauvegarde & Upload ---
        print("\n💾 Sauvegarde et Upload...")
        with tempfile.TemporaryDirectory() as temp_dir_str:
            temp_dir = Path(temp_dir_str)

            # 1. Save Local
            agent.save(temp_dir / "model.zip")

            # 2. Metadata
            hf_folder = f"{grid_size}x{grid_size}/{new_agent_uuid}"
            final_reward = safe_mean([ep["r"] for ep in agent.ep_info_buffer]) if agent.ep_info_buffer else None

            metadata = {
                "uuid": new_agent_uuid,
                "type": "finetuned" if is_finetuning else "fresh",
                "parent_uuid": base_uuid,
                "grid_size": grid_size,
                "algorithm": algorithm,
                "date": readable_date,
                "final_mean_reward": final_reward,
                "hf_folder": hf_folder,
                "mlflow_run_id": run.info.run_id
            }

            with open(temp_dir / "metadata.json", "w") as f:
                json.dump(metadata, f, indent=4)

            # 3. Upload HF
            api = HfApi(token=hf_token)
            api.create_repo(repo_id=hf_repo_id, repo_type="model", exist_ok=True, private=True)
            api.upload_folder(
                folder_path=str(temp_dir),
                path_in_repo=hf_folder,
                repo_id=hf_repo_id,
                commit_message=f"Add {mode_label} model {new_agent_uuid}"
            )

            # 4. Note MLflow
            hf_url = f"https://huggingface.co/{hf_repo_id}/tree/main/{hf_folder}"
            zip_url = f"https://huggingface.co/{hf_repo_id}/resolve/main/{hf_folder}/model.zip?download=true"

            parent_info = f"**Base Modele :** `{base_uuid}`" if base_uuid else ""

            note = textwrap.dedent(f"""\
                ### {mode_label} - Snake {grid_size}x{grid_size}

                ID : {new_agent_uuid}
                Date : {readable_date}
                {parent_info}
                Reward Finale : {final_reward if final_reward else 'N/A'}

                ---
                - [Telecharger Modele]({zip_url})
                - [Voir Fichiers]({hf_url})
                """)

            mlflow.set_tag("mlflow.note.content", note)
            mlflow.set_tag("model_url", hf_url)

            # 5. Model Registry (Logique)
            model_wrapper = SnakeHFModel(repo_id=hf_repo_id, subfolder=hf_folder)
            model_info = mlflow.pyfunc.log_model(artifact_path="snake_model", python_model=model_wrapper)
            mlflow.register_model(model_uri=model_info.model_uri, name=f"Snake_{grid_size}x{grid_size}")

            print(f"✅ Terminé ! Nouveau modèle : {new_agent_uuid}")

    return agent, new_agent_uuid

In [38]:
train_snake(timesteps=100000, grid_size=5, n_envs=8, algorithm="PPO")

In [59]:
train_snake(timesteps=100000, grid_size=5, n_envs=8, algorithm="PPO", base_uuid="20a0ecbd-360b-401c-9977-a6bfaec87e23")


🔍 Scan du dépôt pour trouver l'UUID : 20a0ecbd-360b-401c-9977-a6bfaec87e23 ...
📍 Fichier trouvé : 5x5/20a0ecbd-360b-401c-9977-a6bfaec87e23/metadata.json
📥 Téléchargement du modèle : 5x5/20a0ecbd-360b-401c-9977-a6bfaec87e23/model.zip
✅ Succès ! Agent chargé (Grille 5x5)
✅ Modèle chargé avec succès. Reprise sur grille 5x5.

🚀 Démarrage Run MLflow : FINE-TUNING_20260125_213701_b3293178
🆔 Nouvel ID : b3293178-4918-4855-ab65-4d2980e883ac
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.28     |
|    ep_rew_mean     | 0.109    |
| time/              |          |
|    fps             | 7345     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 131072   |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 6.54       |
|    ep_rew_mean          | 0.328      |
| time/                   |            |
|    fps          

Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (1 / 1): 100%|██████████|  175kB /  175kB,  292kB/s  
Processing Files (1 / 1): 100%|██████████|  175kB /  175kB,  175kB/s  
New Data Upload: 100%|██████████|  175kB /  175kB,  175kB/s  
2026/01/25 21:38:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/25 21:38:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp8jaf9ckt/model, flavor: python_function). Fall back to return ['cloudpickle==3.1.2']. Set logging level to DEBUG to see the full traceback. 
Registered model 'Snake_5x5' already exists. Creating a new version of this model...
Created version '2' of model 'Snake_5x5'.


✅ Terminé ! Nouveau modèle : b3293178-4918-4855-ab65-4d2980e883ac


(<stable_baselines3.ppo.ppo.PPO at 0x728844bd8e20>,
 'b3293178-4918-4855-ab65-4d2980e883ac')

In [56]:
list_snake_models()

🔍 Recherche des modèles dans Lategardener/snake-rl-models...

             MODELE (UUID)                   ||                 DESCRIPTION
--------------------------------------------------------------------------------------------------------------
05x05 / 20a0ecbd-360b-401c-9977-a6bfaec87e23 || Algo: PPO | Date: 25/01/2026 20:58:59 | Reward: -0.06
05x05 / 6ee34c5d-af9e-48a6-b64f-1b1cc8a0080a || Algo: PPO | Date: 25/01/2026 20:41:04 | Reward: 0.01
--------------------------------------------------------------------------------------------------------------
✅ Nombre de modèles trouvés : 2
